<a href="https://colab.research.google.com/github/dspselorm/Recommendation-System-Analysis/blob/main/Recommendation_System_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob
import os

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Merge the two "Item_Properties.csv" files**

In [3]:
# Define the path to your data folder
data_folder = '/content/drive/MyDrive/Azubi_TMP/Recommendation_System_Project/Dataset/Item_properties_data'
items_properties_data = glob.glob(os.path.join(data_folder, "*.csv"))

# Read and combine all CSV files in 2020 folder
df_item = []

for file in items_properties_data:
    try:
        df = pd.read_csv(file)
        df_item.append(df)
        print(f"Loaded: {os.path.basename(file)} with {len(df)} rows")
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Concatenate all DataFrames into one
combined_df = pd.concat(df_item, ignore_index=True)
print(f"Successfully merged {len(df_item)} files into a single DataFrame with {len(combined_df)} rows.")

# Save the combined data
output_path = os.path.join('/content/drive/MyDrive/Azubi_TMP/Recommendation_System_Project/Dataset/', "item_properties_merged.csv")
combined_df.to_csv(output_path, index=False)
print(f"Merged file saved to: {output_path}")

Loaded: item_properties_part1.1.csv with 10999999 rows
Loaded: item_properties_part2.csv with 5638803 rows
Successfully merged 2 files into a single DataFrame with 16638802 rows.
Merged file saved to: /content/drive/MyDrive/Azubi_TMP/Recommendation_System_Project/Dataset/item_properties_merged.csv


### **Upload Datasets**

**1. Item_Properties Dataset**

In [6]:
chunksize = 100000
chunks = []

for chunk in pd.read_csv('/content/drive/MyDrive/Azubi_TMP/Recommendation_System_Project/Dataset/item_properties_merged.csv', chunksize=chunksize):

    chunks.append(chunk)

df_item_properties = pd.concat(chunks, ignore_index=True)
print("File loaded successfully with shape:", df_item_properties.shape)

File loaded successfully with shape: (16638802, 4)


**2. Category_Tree Dataset**

In [7]:
df_category_tree = pd.read_csv('/content/drive/MyDrive/Azubi_TMP/Recommendation_System_Project/Dataset/category_tree.csv')
print("File loaded successfully with shape:", df_category_tree.shape)

File loaded successfully with shape: (1669, 2)


**3. Events Dataset**

In [8]:
df_events = pd.read_csv('/content/drive/MyDrive/Azubi_TMP/Recommendation_System_Project/Dataset/events.csv')
print("File loaded successfully with shape:", df_events.shape)

File loaded successfully with shape: (2756101, 5)


In [9]:
df_category_tree.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
